In [ ]:
'''
Mount Google Drive for COLAB
'''

COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
  %cd gdrive/My\ Drive/Colab\ Notebooks/Benchmark
  !pip install shap

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Benchmark
     |████████████████████████████████| 356 kB 8.4 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491657 sha256=8b6ea49709879ab57775be24ea25806369ed99ec4480f19c1c6f207c9aeb2099
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [ ]:
import pandas as pd
df = pd.read_csv(r'results/Experiments_lstm_model.csv', index_col=['experiment'])
df['mean_loss'] = (df['mae_blind'] + df['rmse_blind'] + df['loss'] + df['mae'] + df['rmse'])/5
display(df.sort_values(['mean_loss']))

,layers,neurons,dropout_rate,activation,timesteps,batch_size,epochs,mae_blind,rmse_blind,loss,mae,rmse,loss.1,mae.1,rmse.1,train_time,mean_loss
experiment,,,,,,,,,,,,,,,,,
3,1,32,0.0,relu,7,30,25,0.161644,0.185609,0.039410,0.027029,0.040088,[[0.23685216]],[[0.16244385]],[[0.24092665]],20.758740,0.090756
77,1,128,0.0,relu,14,20,25,0.159393,0.183446,0.042684,0.031192,0.043333,[[0.25653233]],[[0.18746299]],[[0.26043379]],24.218027,0.092010
32,1,32,0.5,tanh,14,20,25,0.164798,0.189744,0.039818,0.027927,0.040512,[[0.23930526]],[[0.1678407]],[[0.24347428]],22.675887,0.092560
11,1,32,0.0,tanh,7,20,25,0.167542,0.191987,0.039428,0.027163,0.040085,[[0.23696198]],[[0.16324888]],[[0.24090847]],25.650270,0.093241
118,2,32,0.0,tanh,7,10,25,0.154548,0.180119,0.049114,0.037328,0.049475,[[0.29517257]],[[0.22434057]],[[0.29734715]],52.318160,0.094117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2,128,0.0,relu,21,20,25,0.254204,0.279263,0.051496,0.038814,0.051128,[[0.30948997]],[[0.23327164]],[[0.30727759]],35.000761,0.134981
85,1,128,0.0,tanh,14,10,25,0.254173,0.277859,0.056254,0.045654,0.056603,[[0.33808695]],[[0.27438171]],[[0.34018355]],36.904009,0.138109
153,2,64,0.0,relu,21,30,25,0.260263,0.285597,0.057760,0.045044,0.057353,[[0.34713974]],[[0.270713]],[[0.34468896]],28.781191,0.141203


In [ ]:
SEED = 91195003

import load_dataset as ld
import timeseries_preparation as tsp
import experiments as exp
import build_model as bm
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit
import time
import numpy as np

'''
Just load the dataset
'''
df_ph = ld.load_ph_dataset(univariate=True, colab=COLAB)
display(df_ph.sort_values(['value']))

'''
Normalize it
'''
df_data = df_ph.copy()
scalers = tsp.data_normalization(df_data, norm_range=(0, 1))

'''
Set the TimeSeries parameters
'''
multisteps = 2
cv_splits = 3
#must respect the name of the arguments of the build_model function
#timesteps are mandatory
hyperparameters = {
    'layers': [1],
    'neurons': [32],
    'dropout_rate': [0.0],
    'activation': ['relu'],
    'timesteps': [7],
    'batch_size': [30]
}
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=25, min_lr=0.00005),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, mode='min', verbose=0, patience=40)]

param_list, param_names = exp.prepare_param_list(hyperparameters, bm.build_lstm_model)
model = exp.build_model(1, param_names, param_list[0], bm.build_lstm_model, SEED)
print(model.summary())

X, y = tsp.to_supervised(df_data, param_list[0][param_names.index('timesteps')])
tscv = TimeSeriesSplit(n_splits=cv_splits)
for train_index, test_index in tscv.split(X):
  train_idx, val_idx = tsp.split_dataset(train_index, perc=10)
  X_train, y_train = X[train_idx], y[train_idx] 
  X_val, y_val = X[val_idx], y[val_idx] 
  X_test, y_test = X[test_index], y[test_index]
  if callbacks:
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=param_list[0][param_names.index('batch_size')], shuffle=False, verbose=1, callbacks=callbacks)
'''model_json = model.to_json()
print(model_json)
filename = 'models/lstm_' + time.strftime("%Y%m%d%H%M") + '.h5'
model.save_weights(filename)'''

,value
timestep,
2017-11-01,3.550
2016-05-20,4.040
2016-05-04,4.160
2017-11-15,4.165
2017-12-15,4.200
...,...
2019-11-25,8.800
2019-04-11,9.020
2019-11-24,9.040


Model: "lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 7, 1)]            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                1152      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                544       
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
13/13 [==============================] - 2s 50ms/step - loss: 0.5351 - mae: 0.5328 - rmse: 0.5257 - val_

'model_json = model.to_json()\nprint(model_json)\nfilename = \'models/lstm_\' + time.strftime("%Y%m%d%H%M") + \'.h5\'\nmodel.save_weights(filename)'

In [ ]:
def recursive_forecast(model, X_test, y_test, timesteps, multisteps, features, scaler, col_target_position, evaluation_timesteps=30):
    targer_scaler = scaler['value']    
    input_seq = X_test[-evaluation_timesteps:, :, :]
    labels_seq = y_test[-evaluation_timesteps:, :]
    results = list()
    for i in range(len(input_seq)-(multisteps-1)):  #we cannot go through the entire input because we are multistep forecasting!
        inp = input_seq[i].copy()
        print(inp)
        lab = labels_seq[i].copy()
        #for each step of the multistep
        labels = list()
        predictions = list()
        for step in range(1, multisteps+1):
            #reshape
            inp = inp.reshape(1, timesteps, features)
            lab = lab.reshape(1, 1)
            #predict the value for the next timestep
            yhat = model.predict(inp, verbose=0)
            #invert normalized values
            lab_inversed = targer_scaler.inverse_transform(lab)
            yhat_inversed = targer_scaler.inverse_transform(yhat)
            #print(lab_inversed, yhat_inversed)
            #store results
            labels.append(lab_inversed[0][0])
            predictions.append(yhat_inversed[0][0])
            #insert a new value into the input sequence to predict the next timestep.
            if step != multisteps:
                #add yhat to the input sequence
                new_line = input_seq[i+step][-1].copy()
                np.put(new_line, col_target_position, yhat)
                new_line = new_line.reshape(1, features)
                inp = np.concatenate((inp[0], new_line))
                inp = inp[-timesteps:]
                #update label to the next timestep
                lab = labels_seq[i+step]
        results.append((np.array(predictions)))
    return np.array(results)

#generated_data = np.random.uniform(low=6, high=9, size=9)
generated_data = [7.61, 7.075, 7.525, 6.9849, 7.105, 7.185, 7.04, 7.15, 7.165, 7.06]
generated_df = pd.DataFrame(generated_data, columns=['value'])
generated_df['value'] = scalers['value'].transform(generated_df[['value']])
X_generated, y_generated = tsp.to_supervised(generated_df , param_list[0][param_names.index('timesteps')])
generated_df['ph'] = scalers['value'].inverse_transform(generated_df[['value']])
display(generated_df)
print(recursive_forecast(model, X_generated, y_generated, param_list[0][param_names.index('timesteps')], 2, 1, scalers, generated_df.columns.get_loc('value')))

,value,ph
0,0.675541,7.6100
1,0.586522,7.0750
2,0.661398,7.5250
3,0.571531,6.9849
4,0.591514,7.1050
5,0.604825,7.1850
6,0.580699,7.0400
7,0.599002,7.1500
8,0.601498,7.1650
9,0.584027,7.0600


[[0.67554075]
 [0.58652246]
 [0.6613977 ]
 [0.57153076]
 [0.59151417]
 [0.6048253 ]
 [0.58069885]]
[[0.58652246]
 [0.6613977 ]
 [0.57153076]
 [0.59151417]
 [0.6048253 ]
 [0.58069885]
 [0.59900165]]
[[7.163911  7.143458 ]
 [7.140236  7.1433816]]
